# 计算智能作业1-报告

**运行环境：**
+ python3
+ pandas
+ numpy
+ collections

## 1. 数据
文件iris中包含了iris数据，其中每行的前四个数据代表一个样本，最后一个数据表示该样本的类别。

In [25]:
import pandas as pd
dataset = pd.read_csv("./iris.dat", delimiter='\t', names=['feat0', 'feat1', 'feat2', 'feat3', 'label'])
print("数据预览 \n")
dataset

数据预览 



,feat0,feat1,feat2,feat3,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## 2. C_means聚类
**要求**：用C_means的方法对iris数据做聚类，要求聚成3类。
**给出下列数据**：
+ 初始类中心点
+ 迭代次数
+ 聚类结果（每类包含的样本，类中心）
+ 错误率


### 2.1 预处理
1. 导入必要的库
2. 预定义变量

In [26]:
import numpy as np
import random
import collections

# predefine  variables
cluster_num = 3

# load data 
dataset = np.loadtxt("./iris.dat")
samples = dataset[:, 0:4].astype(np.float64)
labels  = dataset[:, 4].astype(np.int32)
samples_num = samples.shape[0]

### 2.2 聚类过程

1） 初始化聚类中心

In [27]:
choice = np.random.choice(samples_num, cluster_num)
centers = samples[choice]

print("初始化类中心为：")
for center in centers:
    print(str(center))

初始化类中心为：
[6.  2.2 5.  1.5]
[5.7 2.8 4.1 1.3]
[6.7 3.3 5.7 2.1]


2） 迭代

In [28]:
c_label = np.zeros(labels.shape, dtype=np.int32)
clusters = [set() for _ in range(cluster_num)]

it = 0
while it < 100:
    print("iteartion : %d" %it)
    clusters_ = [set() for _ in range(cluster_num)]
    
    # start cluster 开始聚类
    for i in range(samples_num):
        sample = samples[i]
        parallel = np.tile(sample, (cluster_num, 1))

        # It's no need to calculate the sqrt
        distances = np.sum(np.square(parallel - centers), axis=1)
        minidx = distances.argmin()

        # Put into box and update label
        clusters_[minidx].add(i)
        c_label[i] = minidx

    # recalculate centers 更新类中心点
    static = True
    for i in range(cluster_num):
        c, c_ = clusters[i], clusters_[i]
        if c == c_:
            continue
        else:
            static = False
            # renew center
            samples_c = samples[list(c_)]
            centers[i] = np.average(samples_c, axis=0)
    
    clusters = clusters_
    if static:
        break
    it += 1

print("迭代次数：%d" %(it+1))

iteartion : 0
iteartion : 1
iteartion : 2
iteartion : 3
iteartion : 4
iteartion : 5
迭代次数：6


### 2.3 聚类结果

1) 聚类结果

In [29]:
for cluster, center in zip(clusters, centers):
    print("--------------------------------------------------------")
    print("类中心 ： %s" %str(center))
    print("类中元素：" )
    print(cluster)
    print("类元素个数：%d" %len(cluster))

--------------------------------------------------------
类中心 ： [5.9016129  2.7483871  4.39354839 1.43387097]
类中元素：
{133, 138, 142, 146, 149, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 106, 113, 114, 119, 121, 123, 126, 127}
类元素个数：62
--------------------------------------------------------
类中心 ： [5.006 3.428 1.462 0.246]
类中元素：
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49}
类元素个数：50
--------------------------------------------------------
类中心 ： [6.85       3.07368421 5.74210526 2.07105263]
类中元素：
{128, 129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 143, 144, 145, 147, 148, 52, 77, 100, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 115, 116, 117, 118, 120, 122, 124, 125}
类元素个数：38


2) 错误率

In [30]:
true_num = 0
for cluster in clusters:
    label_dict = {}
    for it in cluster:
        try:
            label_dict[labels[it]] += 1
        except KeyError:
            label_dict[labels[it]] = 1
    true_num += max(label_dict.values())

print("聚类错误率： %f" %(1 - true_num/len(labels)))

聚类错误率： 0.106667


## 3. 谱聚类
用谱聚类方法对iris数据作聚类

**算法流程：**
1. 构造相似性矩阵 W 和度矩阵 D
2. 计算拉普拉斯矩阵 L
3. 构建标准化之后的拉普拉斯矩阵 D^(-1/2)LD^(-1/2)
4. 计算 D^(-1/2)LD^(-1/2)最小的k1个特征值所各自对应的特征向量f
5. 将各自对应的特征向量f组成的矩阵按行标准化，最终组成 n*k1 维的特征矩阵F
6. 对F中的每一行作为一个k1维的样本，共n个样本，用K-means方法聚类
7. 得到簇划分 C( c1, c2, ... ck2)

### 3.1 计算邻接矩阵

In [31]:
sigma = 1
rows = len(samples)

W = np.zeros((rows, rows))
for i in range(rows):
    for j in range(rows):
        if (i != j):
            W[i][j] = np.exp(-np.sum((samples[i] - samples[j])**2/(2*sigma**2)))

print("邻接矩阵 W 为：")
W

邻接矩阵 W 为：


array([[0.00000000e+00, 8.65022293e-01, 8.78095431e-01, ...,
        4.79668697e-05, 2.00957944e-05, 1.89712650e-04],
       [8.65022293e-01, 0.00000000e+00, 9.55997482e-01, ...,
        4.02661255e-05, 1.46656872e-05, 1.79560205e-04],
       [8.78095431e-01, 9.55997482e-01, 0.00000000e+00, ...,
        1.91157109e-05, 7.84997666e-06, 9.70775902e-05],
       ...,
       [4.79668697e-05, 4.02661255e-05, 1.91157109e-05, ...,
        0.00000000e+00, 8.26959134e-01, 8.14647316e-01],
       [2.00957944e-05, 1.46656872e-05, 7.84997666e-06, ...,
        8.26959134e-01, 0.00000000e+00, 7.44531587e-01],
       [1.89712650e-04, 1.79560205e-04, 9.70775902e-05, ...,
        8.14647316e-01, 7.44531587e-01, 0.00000000e+00]])

### 3.2 计算标准化拉普拉斯矩阵


In [32]:
# 度矩阵
degree_matrix = np.sum(W, axis=1)

L = np.diag(degree_matrix) - W

# 拉普拉斯矩阵标准化，Ncut切图
sqrt_degree_matrix = np.diag(1.0 / (degree_matrix**0.5)) # D^(-1/2)
L = np.dot(np.dot(sqrt_degree_matrix, L), sqrt_degree_matrix) # D^(-1/2)*L*D^(-1/2)

print("标准化拉普拉斯矩阵 L：")
L

标准化拉普拉斯矩阵 L：


array([[ 1.00000000e+00, -2.09800762e-02, -2.12397985e-02, ...,
        -9.94628311e-07, -4.70407683e-07, -3.93702551e-06],
       [-2.09800762e-02,  1.00000000e+00, -2.39326914e-02, ...,
        -8.64142566e-07, -3.55302089e-07, -3.85663154e-06],
       [-2.12397985e-02, -2.39326914e-02,  1.00000000e+00, ...,
        -4.09133391e-07, -1.89667371e-07, -2.07943791e-06],
       ...,
       [-9.94628311e-07, -8.64142566e-07, -4.09133391e-07, ...,
         1.00000000e+00, -1.71284897e-02, -1.49591654e-02],
       [-4.70407683e-07, -3.55302089e-07, -1.89667371e-07, ...,
        -1.71284897e-02,  1.00000000e+00, -1.54337212e-02],
       [-3.93702551e-06, -3.85663154e-06, -2.07943791e-06, ...,
        -1.49591654e-02, -1.54337212e-02,  1.00000000e+00]])

### 3.3 特征值分解和排序


In [41]:
# 特征值分解
lam, H = np.linalg.eig(L)

# 特征值排序
t = np.argsort(lam)
H = np.c_[H[:, t[0]], H[:, t[1]], H[:, t[2]]]

# 归一化处理
sum = np.sqrt(np.sum(H**2, axis=1, keepdims=True))
H = H/sum

### 3.4 KMeans聚类

In [42]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3)
model.fit(H)

KMeans(n_clusters=3)

### 3.5 结果分析
**聚类结果:**

In [43]:
labels = model.labels_
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0])

**错误率：**

In [44]:
true_label = 0
c1 = collections.Counter(labels[0:50])
true_label += c1.most_common(1)[0][1]

c2 = collections.Counter(labels[50:100])
true_label += c2.most_common(1)[0][1]

c3 = collections.Counter(labels[100:150])
true_label += c3.most_common(1)[0][1]

print("谱聚类错误率： %f" %(1 - true_label / len(labels)))

谱聚类错误率： 0.106667
